<a href="https://colab.research.google.com/github/s-c-soma/cmpe297_BreastCancerDiagnosis_withDeepTransferLearning/blob/main/tfx_pipeline_colab/TFX_Colab_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENV Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow_model_analysis
!pip install tfx
!pip install tflite-support

     |████████████████████████████████| 1.7MB 6.4MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 17.7MB 293kB/s 
     |████████████████████████████████| 788kB 35.4MB/s 
     |████████████████████████████████| 9.6MB 36.7MB/s 
     |████████████████████████████████| 2.3MB 34.4MB/s 
     |████████████████████████████████| 368kB 39.3MB/s 
     |████████████████████████████████| 829kB 33.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 153kB 50.9MB/s 
     |████████████████████████████████| 2.3MB 33.8MB/s 
     |████████████████████████████████| 276kB 29.7MB/s 
     |████████████████████████████████| 92kB 11.2MB/s 
     |████████████████████████████████| 440kB 37.9MB/s 
     |████████████████████████████████| 184kB 47.3MB/s 
     |████████████████████████████████| 153kB 48.9MB/s 
     |████████████████████████████████| 174kB 50.0MB/s 
     |████████████████████████████████| 256kB 49.5MB/s

     |████████████████████████████████| 2.3MB 6.6MB/s 
     |████████████████████████████████| 1.4MB 27.4MB/s 
     |████████████████████████████████| 2.9MB 38.2MB/s 
     |████████████████████████████████| 1.2MB 44.5MB/s 
     |████████████████████████████████| 112kB 42.7MB/s 
     |████████████████████████████████| 1.7MB 38.2MB/s 
     |████████████████████████████████| 409kB 38.8MB/s 
     |████████████████████████████████| 2.3MB 40.9MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 153kB 46.6MB/s 
     |████████████████████████████████| 1.5MB 35.0MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 296kB 38.4MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp37-none-any.whl size=73200 sha256=ee46c2d49e78c7a2d854f5565a5e9af33e40f4017906f1943f6d7ac356654d5f
  Store

In [1]:
!gdown --id 1M6mpP_PjReXXY62hUn0t2JXR8c-Uinkk 

Downloading...
From: https://drive.google.com/uc?id=1M6mpP_PjReXXY62hUn0t2JXR8c-Uinkk
To: /content/combined.zip
20.6MB [00:00, 78.0MB/s]


In [2]:
import zipfile
with zipfile.ZipFile('/content/combined.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/combined')

In [3]:
!ls /content/combined/12819

test  train


In [4]:
from PIL import Image
import os, sys

path = "/content/combined/12819/train/1/"
dirs = os.listdir( path )
 
try:
  os.mkdir('/content/converted')
  os.mkdir('/content/converted/train/')
  os.mkdir('/content/converted/test/')
  os.mkdir('/content/converted/train/1')
  os.mkdir('/content/converted/train/0')
  os.mkdir('/content/converted/test/1')
  os.mkdir('/content/converted/test/0')

except:
  pass

def resize(p1='/content/combined/12819/',p2='train/1'):
    dirs = os.listdir( p1 + p2)
    for item in dirs:
        if os.path.isfile(p1 + p2+item):
            im = Image.open(p1 + p2+item)
            f, e = os.path.splitext(p1 + p2 +item)
            # print(item)
            imResize = im.resize((224,224), Image.ANTIALIAS)
            imResize.save('/content/converted/'+ p2 + item , quality=90)

resize(p2='train/1/')
resize(p2='train/0/')
resize(p2='test/1/')
resize(p2='test/0/')


In [5]:
from __future__ import absolute_import, division, print_function
 
import tensorflow as tf
print(tf.version)
  
# tf.enable_eager_execution()
 
AUTOTUNE = tf.data.experimental.AUTOTUNE
 
import IPython.display as display
import matplotlib.pyplot as plt

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [6]:
tf.config.run_functions_eagerly(True)


In [7]:
import numpy as np, pandas as pd, os
import matplotlib.pyplot as plt, cv2
import tensorflow as tf, re, math
import glob

In [8]:
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory

# !unzip /content/drive/My\ Drive/garbage-classification.zip


def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  

cwd = "/content/converted/train"

recordPath = "/content/"

bestNum = 1000

num = 0

recordFileNum = 0

keys = [str(i) for i in list(range(10))]
values = [i for i in list(range(10))]

classes ={'0': 0, '1': 1} 

# print(classes)

recordFileName = ("train.tfrecords-%.3d" % recordFileNum)

writer = tf.io.TFRecordWriter(recordPath + recordFileName)

import cv2
# print("Creating the 000 tfrecord file")
for name, label in classes.items():
    # print(name)
    # print(label)
    class_path = os.path.join(cwd, name)
    # print(class_path)
    for img_name in os.listdir(class_path):

      img_path = os.path.join(class_path, img_name)
      image_string = open(img_path, 'rb').read()
    
      image_shape = tf.image.decode_jpeg(image_string).shape
      # print(image_shape)
      


      feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image': _bytes_feature(image_string),
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(example.SerializeToString())
writer.close()

In [9]:
import os  # used for directory operations
import tensorflow as tf
from PIL import Image  # used to read images from directory

# !unzip /content/drive/My\ Drive/garbage-classification.zip


def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  

cwd = "/content/converted/test"

recordPath = "/content/"

bestNum = 1000

num = 0

recordFileNum = 0

keys = [str(i) for i in list(range(10))]
values = [i for i in list(range(10))]

classes ={'0': 0, '1': 1} 

# print(classes)

recordFileName = ("test.tfrecords-%.3d" % recordFileNum)

writer = tf.io.TFRecordWriter(recordPath + recordFileName)

import cv2
# print("Creating the 000 tfrecord file")
for name, label in classes.items():
    # print(name)
    # print(label)
    class_path = os.path.join(cwd, name)
    # print(class_path)
    for img_name in os.listdir(class_path):

      img_path = os.path.join(class_path, img_name)
      image_string = open(img_path, 'rb').read()
    
      image_shape = tf.image.decode_jpeg(image_string).shape
      # print(image_shape)
      


      feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image': _bytes_feature(image_string),
      }

      example = tf.train.Example(features=tf.train.Features(feature=feature))
      writer.write(example.SerializeToString())
writer.close()

In [10]:
import os
try:
  os.mkdir('breast_cancer')
  os.mkdir('breast_cancer/data')
  os.mkdir('breast_cancer/data/train')
  os.mkdir('breast_cancer/data/test')
  os.mkdir('breast_cancer/model') #scs
except:
  pass

In [11]:
!cp /content/train.tfrecords-000 /content/breast_cancer/data/train
!cp /content/test.tfrecords-000 /content/breast_cancer/data/test

In [12]:
!gdown --id 15vX_nlCP36SSv1YqIrpQFZAoBrR7fG8K
!cat /content/utils_native_keras.py 

Downloading...
From: https://drive.google.com/uc?id=15vX_nlCP36SSv1YqIrpQFZAoBrR7fG8K
To: /content/utils_native_keras.py
100% 15.6k/15.6k [00:00<00:00, 25.1MB/s]
# Lint as: python2, python3
# Copyright 2019 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Python source file includes CIFAR10 utils for Keras model.

The utilities in this file are used to build a model with native Keras.
This module file will be used in Transform and generic Trainer.
"""

from __fu

In [13]:
!cp /content/utils_native_keras.py /content/breast_cancer/
!cat /content/breast_cancer/utils_native_keras.py 

# Lint as: python2, python3
# Copyright 2019 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Python source file includes CIFAR10 utils for Keras model.

The utilities in this file are used to build a model with native Keras.
This module file will be used in Transform and generic Trainer.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from typing import List, Text
import absl
import te

In [14]:
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.components import ModelValidator
import absl
import tensorflow_model_analysis as tfma

from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import ImportExampleGen
from tfx.components import Pusher
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input


import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input


%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

# TFX Pipeline

In [15]:
context = InteractiveContext()


In [16]:
try:
  os.mkdir('tfx_whatever')
except:
  pass

In [17]:
from pathlib import Path
_tfx_ = "/content/tfx_pipeline"
#home = str(Path)
_pipeline_name = 'waste_mgm_beam'

HOME=os.environ.get
_breast_cancer_root = os.path.join(os.environ['HOME'], 'breast_cancer')
# _cifar10_root = os.path.join(_tfx_, 'waste_mgm')
print(_breast_cancer_root)
_data_root = os.path.join(_breast_cancer_root, 'data')
_module_file = os.path.join(_breast_cancer_root, 'utils_native_keras.py')
# _serving_model_dir = os.path.join(_breast_cancer_root, 'serving_model',
#                                   _pipeline_name)
_serving_model_dir = os.path.join('breast_cancer/model', 'serving_model',_pipeline_name) #_cifar10_root #scs
print(_data_root)

/root/breast_cancer
/root/breast_cancer/data


In [18]:
_tfx_dir = "/content/tfx_pipeline"
_tfx_root = os.path.join(_tfx_dir, 'tfx')
print(_tfx_root)

/content/tfx_pipeline/tfx


In [19]:
_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)

_metadata_path = os.path.join(_tfx_root, 'metadata', _pipeline_name,
                              'metadata.db')

In [20]:
_module_file = '/content/breast_cancer/utils_native_keras.py'
_data_root = '/content/breast_cancer/data/train'
examples = external_input(_data_root)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


In [21]:
pipeline_name=_pipeline_name
pipeline_root=_pipeline_root
data_root=_data_root
module_file=_module_file
serving_model_dir=_serving_model_dir
metadata_path=_metadata_path

In [22]:
context = InteractiveContext()


## 1.ExampleGen

In [23]:
examples = external_input(_data_root)

  # Brings data into the pipeline.
example_gen = ImportExampleGen(input=examples)

In [24]:
context.run(example_gen)


ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/ImportExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:100960532,xor_checksum:1620673902,sum_checksum:1620673902"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## 2.StatisticsGen

In [25]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [26]:
context.show(statistics_gen.outputs['statistics'])


## 3.SchemaGen

In [27]:
schema_gen = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [28]:
context.show(schema_gen.outputs['schema'])


,Type,Presence,Valency,Domain
Feature name,,,,
'depth',INT,required,,-
'height',INT,required,,-
'image',BYTES,required,,-
'label',INT,required,,-
'width',INT,required,,-


## 4.ExampleValidator


In [29]:
example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [30]:
context.show(example_validator.outputs['anomalies'])


/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## 5.Transform




In [31]:
transform = Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_gen.outputs['schema'],
      module_file=module_file)

In [32]:
context.run(transform)


Instructions for updating:
Use fn_output_signature instead
Instructions for updating:
Use ref() instead.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/transform_graph/5/.temp_path/tftransform_tmp/be4e26679b2147b8a84f1603a8b2e5bf/saved_model.pb


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [33]:
transform.outputs


{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(artifact: id: 5
type_id: 13
uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/transform_graph/5"
custom_properties {
  key: "name"
  value {
    string_value: "transform_graph"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "0.29.0"
  }
}
state: LIVE
, artifact_type: id: 13
name: "TransformGraph"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 6
type_id: 5
uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Transform/transformed_examples/5"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
 

## 6.Trainer


In [34]:
!gdown --id 15vX_nlCP36SSv1YqIrpQFZAoBrR7fG8K
!cat /content/utils_native_keras.py 
!cp /content/utils_native_keras.py /content/breast_cancer/
!cat /content/breast_cancer/utils_native_keras.py 

Downloading...
From: https://drive.google.com/uc?id=15vX_nlCP36SSv1YqIrpQFZAoBrR7fG8K
To: /content/utils_native_keras.py
100% 15.6k/15.6k [00:00<00:00, 25.3MB/s]
# Lint as: python2, python3
# Copyright 2019 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Python source file includes CIFAR10 utils for Keras model.

The utilities in this file are used to build a model with native Keras.
This module file will be used in Transform and generic Trainer.
"""

from __fu

In [35]:
from tfx.components import Trainer
from tfx.components.trainer.executor import Executor
from tfx.dsl.components.base import executor_spec

# trainer = Trainer(
#       module_file=module_file,
#       custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
#       examples=transform.outputs['transformed_examples'],
#       schema=schema_gen.outputs['schema'],
#       transform_graph=transform.outputs['transform_graph'],
#       train_args=trainer_pb2.TrainArgs(num_steps=1000),
#       eval_args=trainer_pb2.EvalArgs(num_steps=500))
_labels_path = os.path.join(_data_root, 'labels.txt')
trainer = Trainer(
      module_file=module_file,
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=transform.outputs['transformed_examples'],
      schema=schema_gen.outputs['schema'],
      transform_graph=transform.outputs['transform_graph'],
      train_args=trainer_pb2.TrainArgs(num_steps=150),
      eval_args=trainer_pb2.EvalArgs(num_steps=1),
      custom_config={'labels_path': _labels_path})

In [36]:
context.run(trainer)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


58892288/58889256 [==============================] - 1s 0us/step
Epoch 1/30
4/4 [==============================] - 34s 3s/step - loss: 0.5571 - sparse_categorical_accuracy: 0.7323 - val_loss: 0.3081 - val_sparse_categorical_accuracy: 0.9375
Epoch 2/30
4/4 [==============================] - 7s 2s/step - loss: 0.4085 - sparse_categorical_accuracy: 0.8635 - val_loss: 0.4228 - val_sparse_categorical_accuracy: 0.8438
Epoch 3/30
4/4 [==============================] - 6s 2s/step - loss: 0.4104 - sparse_categorical_accuracy: 0.8677 - val_loss: 0.2814 - val_sparse_categorical_accuracy: 0.9375
Epoch 4/30
4/4 [==============================] - 7s 2s/step - loss: 0.4775 - sparse_categorical_accuracy: 0.8115 - val_loss: 0.2803 - val_sparse_categorical_accuracy: 0.9375
Epoch 5/30
4/4 [==============================] - 7s 2s/step - loss: 0.3674 - sparse_categorical_accuracy: 0.8646 - val_loss: 0.4798 - val_sparse_categorical_accuracy: 0.7812
Epoch 6/30
4/4 [==============================] - 7s 2s/ste

ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## 7.Evaluator


In [37]:
 # Get the latest blessed model for model validation.
model_resolver = ResolverNode(
    instance_name='latest_blessed_model_resolver',
    resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

ExecutionResult(
    component_id: Resolver.latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [38]:
# Uses TFMA to compute evaluation statistics over features of a model and
# perform quality validation of a candidate model (compare to a baseline).
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label_xf', model_type='tf_lite')], #tf_lite
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(
                class_name='SparseCategoricalAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.55}),
                    # Change threshold will be ignored if there is no
                    # baseline model resolved from MLMD (first run).
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value': -1e-3})))
        ])
    ])

In [39]:
# Uses TFMA to compute the evaluation statistics over features of a model.
# We evaluate using the materialized examples that are output by Transform
# because
# 1. the decoding_png function currently performed within Transform are not
# compatible with TFLite.
# 2. MLKit requires deserialized (float32) tensor image inputs
# Note that for deployment, the same logic that is performed within Transform
# must be reproduced client-side.
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

ERROR:absl:There are change thresholds, but the baseline is missing. This is allowed only when rubber stamping (first run).


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 10
        type_id: 20
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Evaluator/evaluation/8"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 11
        type_id: 21
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Evaluator/blessing/8"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Trainer/model/6"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 8
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 21
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

## 8.Pusher


In [40]:
file_path = '/content/drive/Shared drives/258 Project/258 Project/code/tfx_experiment/'

In [41]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(),'serving_model/breas_cancer')
_serving_model_dir_lite = file_path#os.path.join(tempfile.mkdtemp(),'serving_model/breas_cancer')

In [42]:

# Checks whether the model passed the validation steps and pushes the model
# to a file destination if check passed.
# _serving_model_dir_lite = os.path.join(_cifar10_root, 'serving_model_lite',
#                                        _pipeline_name)
#scs
test = "test"
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            # base_directory=_serving_model_dir_lite)))
            base_directory=test))) #_serving_model_dir_lite #scs
context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 12
        type_id: 23
        uri: "/tmp/tfx-interactive-2021-05-10T19_11_46.505196-ilkgehsq/Pusher/pushed_model/9"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "test/1620674462"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1620674462"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "0.29.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [43]:
serving_model_dir

'breast_cancer/model/serving_model/waste_mgm_beam'

# Converting .tflite to .pb

In [44]:
model_path = 'breast_cancer/model/serving_model/waste_mgm_beam'

In [45]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_path+"tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)